In [1]:
import cv2
import numpy as np
import dlib
from math import hypot   #this library helps to do calculations easily


cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def midpoint(p1, p2):  #returns the midpoint
    return int((p1.x + p2.x) / 2), int((p1.y + p2.y)/ 2)  #pixel canot be in float,so find integer.

def get_gaze_ratio(eye_points, landmarks):
    
        #Gaze detection - drawing polygon on the left eye and getting the exact region of the eye
        left_eye_region = np.array([(landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y),
                                   (landmarks.part(eye_points[1]).x, landmarks.part(eye_points[1]).y),
                                   (landmarks.part(eye_points[2]).x, landmarks.part(eye_points[2]).y),
                                   (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y),
                                   (landmarks.part(eye_points[4]).x, landmarks.part(eye_points[4]).y),
                                   (landmarks.part(eye_points[5]).x, landmarks.part(eye_points[5]).y)], np.int32)
        #drawing polygon on the left eye
        #cv2.polylines(frame, [left_eye_region], True, (0,0,255), 2)
        
        
        height, width, _ = frame.shape
        mask = np.zeros((height, width), np.uint8)  #this is a black screen of size same as our frame
        #We will create the eye region in this mask
        cv2.polylines(mask, [left_eye_region], True, (255,255,255), 2)  #creating the polygon of the eye region
        cv2.fillPoly(mask, [left_eye_region], 255)  #filling the polygon
        #mask gives exact eye, so we put this mask over the frame so as to get exact eye and the rest extra portion of eye is removed.
        eye = cv2.bitwise_and(gray, gray, mask = mask)
        
        
        #detecting the eye region
        min_x = np.min(left_eye_region[:, 0])  #left most pt is min x
        max_x = np.max(left_eye_region[:, 0])  #rightt most pt is max x
        min_y = np.min(left_eye_region[:, 1])  #top most pt is min y
        max_y = np.max(left_eye_region[:, 1])  #bottomt most pt is max y
        
        gray_eye = eye[min_y : max_y, min_x : max_x]
        _, threshold_eye = cv2.threshold(gray_eye, 40, 255, cv2.THRESH_BINARY)
        
        #dividing the threshold into 2 parts - left and write threshold
        height, width = threshold_eye.shape
        left_side_threshold = threshold_eye[0: height, 0: int(width/2)]
        right_side_threshold = threshold_eye[0: height, int(width/2): width]
        
        #calculating the amount of white part in both the thresholds - left and right
        left_side_white = cv2.countNonZero(left_side_threshold)
        right_side_white = cv2.countNonZero(right_side_threshold)
        
        #handling zero divison error
        if left_side_white == 0:
            gaze_ratio = 1   #means we are looking at the left
        elif right_side_white == 0:
            gaze_ratio = 5   #means we are looking at the right
        else:
            gaze_ratio = left_side_white/ right_side_white
        

        return gaze_ratio
    
    

font = cv2.FONT_HERSHEY_PLAIN

def get_blinking_ratio(eye_points, landmarks):    #eye_points is an array of points
    left_point = (landmarks.part(eye_points[0]).x, landmarks.part(eye_points[0]).y)
    right_point = (landmarks.part(eye_points[3]).x, landmarks.part(eye_points[3]).y)
    center_top = midpoint(landmarks.part(eye_points[1]), landmarks.part(eye_points[2]))
    center_bottom = midpoint(landmarks.part(eye_points[5]), landmarks.part(eye_points[4]))
        
    #hor_line = cv2.line(frame, left_point, right_point, (0,255,0), 2)
    #ver_line = cv2.line(frame, center_top, center_bottom, (0,255,0), 2)
        
    #checking blinking of line
    ver_line_length = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))
    hor_line_length = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ratio = hor_line_length/ ver_line_length
    
    return ratio

while True:
    
    _, frame = cap.read()
    #showing direction when we turn the face eithre left and right
    new_frame = np.zeros((500, 500, 3), np.uint8)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        #print(face)
        x, y = face.left(), face.top()
        x1, y1 = face.right(), face.bottom()
        #cv2.rectangle(frame, (x, y), (x1, y1), (0,255,0), 2)
        
        landmarks = predictor(gray, face)   #landmark is an object.
        
        #Detect blinking
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        if blinking_ratio > 5.7:
            cv2.putText(frame, "BLINKING", (50,150), font, 3,(255, 0, 0))
        
        
        #gaze detection
        gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
        gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
        gaze_ratio = (gaze_ratio_left_eye + gaze_ratio_right_eye)/ 2
        
        if gaze_ratio < 0.8: 
            cv2.putText(frame, "RIGHT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (0, 0, 255)
        elif 0.8 < gaze_ratio < 1.2:
            cv2.putText(frame, "CENTER", (50, 100), font, 2, (0,0,255), 3)
        else:
            cv2.putText(frame, "LEFT", (50, 100), font, 2, (0,0,255), 3)
            new_frame[:] = (255, 0, 0)
        
        
        
        
        
        
        
    cv2.imshow("Frame", frame)
    cv2.imshow('New Frame', new_frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()